In [106]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista
import ufl
import time
from mpi4py import MPI
from petsc4py import PETSc
# import dolfinx
from dolfinx import fem, mesh, plot, nls, log, io
from dolfinx import cpp as _cpp
import meshio
import os
%load_ext autoreload
%autoreload 2

# print(dolfinx.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
from utils import *
from model import *

In [108]:
# Slab domain (single capillary)
vertex = [8,8,120]
num_nodes = [8,8,60]
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], vertex], num_nodes, mesh.CellType.tetrahedron) 

In [109]:
# plot_mesh(domain)

In [110]:
results_folder = r"../../results-data/"
exp_folder = "slab_test"

In [111]:
model = PerfusionGasExchangeModel(mesh_path=None,
                                  results_path=results_folder,
                                  exp_path=exp_folder,
                                  params=None)

In [112]:
model.Setup(domain)
model.parameter_setup()

max_dims = [  8.   8. 120.]
min_dims = [0. 0. 0.]


In [113]:
# p, u = model.Perfusion(domain, plot=True, save=True)

In [114]:
model.results_path

'../../results-data/slab_test'

In [115]:
# model.GasExchange(domain, guess=None, save=True, plot=False)

In [116]:
domain.topology.dim

3

In [117]:
P1 = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)
V = fem.FunctionSpace(domain, P1*P1)
p_XY = fem.Function(V)
p_O2, p_CO2 = p_XY.split()

In [118]:
ds = ufl.Measure('ds', domain=domain, 
                 subdomain_data=model.facet_tag, 
                 metadata={'quadrature_degree': 2})

In [119]:
V

FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False), (3,)), 4), MixedElement(FiniteElement('Lagrange', tetrahedron, 1), FiniteElement('Lagrange', tetrahedron, 1)))

In [120]:
V.sub(0)

FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False), (3,)), 4), FiniteElement('Lagrange', tetrahedron, 1))

In [121]:
V.sub(1)

FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False), (3,)), 4), FiniteElement('Lagrange', tetrahedron, 1))

In [122]:
topology, cells, geometry = plot.vtk_mesh(domain, domain.topology.dim)

max_dims = np.around([max(geometry[:,0]), max(geometry[:,1]), max(geometry[:,2])], 5)
min_dims = np.around([min(geometry[:,0]), min(geometry[:,1]), min(geometry[:,2])], 5)

print(f"max_dims = {max_dims}")
print(f"min_dims = {min_dims}")

max_dims = [  8.   8. 120.]
min_dims = [0. 0. 0.]


In [157]:
atol = 1E-10
def right(x):
    return np.isclose(x[0], min_dims[0], atol=atol)
def left(x):
    return np.isclose(x[0], max_dims[0], atol=atol)
def top(x):
    return np.isclose(x[1], min_dims[1], atol=atol)
def bottom(x):
    return np.isclose(x[1], max_dims[1], atol=atol)
def front(x):
    return np.isclose(x[2], min_dims[2], atol=atol)
def back(x):
    return np.isclose(x[2], max_dims[2], atol=atol)
def all(x):
    return np.full(x.shape[1], True, dtype=bool)
def air(x):
    return np.logical_and(all(x), np.logical_not(np.logical_or(front(x), back(x))))


fdim = domain.topology.dim -1

right_facets = mesh.locate_entities_boundary(domain, fdim, right)
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
top_facets = mesh.locate_entities_boundary(domain, fdim, top)
bottom_facets = mesh.locate_entities_boundary(domain, fdim, bottom)
front_facets = mesh.locate_entities_boundary(domain, fdim, front)
back_facets = mesh.locate_entities_boundary(domain, fdim, back)
air_facets = mesh.locate_entities_boundary(domain, fdim, air)


marked_facets = np.hstack([right_facets, left_facets, top_facets, bottom_facets, front_facets, back_facets, air_facets])
marked_values = np.hstack([np.full_like(right_facets, 1), np.full_like(left_facets, 2),
                            np.full_like(top_facets, 3), np.full_like(bottom_facets, 4),
                            np.full_like(front_facets, 5), np.full_like(back_facets, 6),
                            np.full_like(air_facets, 7)])

sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, model.fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

In [158]:
back_facets

array([44596, 44597, 44598, 44824, 44825, 45019, 45026, 45027, 45057,
       45058, 45360, 45361, 45374, 45375, 45570, 45571, 45584, 45585,
       45746, 45753, 45754, 45808, 45809, 45866, 45867, 46048, 46049,
       46076, 46077, 46114, 46115, 46229, 46230, 46257, 46258, 46295,
       46379, 46386, 46387, 46441, 46442, 46513, 46514, 46635, 46636,
       46663, 46664, 46719, 46787, 46788, 46815, 46816, 46912, 46919,
       46920, 46974, 46975, 47024, 47025, 47122, 47123, 47150, 47151,
       47194, 47246, 47247, 47274, 47275, 47347, 47354, 47355, 47404,
       47405, 47437, 47438, 47511, 47512, 47539, 47540, 47571, 47607,
       47608, 47635, 47636, 47684, 47691, 47692, 47724, 47725, 47752,
       47753, 47802, 47803, 47823, 47824, 47850, 47870, 47871, 47891,
       47892, 47923, 47930, 47931, 47941, 47942, 47962, 47963, 47995,
       47996, 48004, 48005, 48024, 48035, 48036, 48044, 48045, 48065,
       48066, 48074, 48075, 48095, 48096, 48103, 48110, 48111, 48119,
       48120, 48127]

In [160]:
# Filter out ghosted cells
num_cells_local = mesh.locate_entities_boundary(domain, fdim, all)
print(f"num_cells_local = {num_cells_local}")
marker = np.zeros(np.max(num_cells_local)+1, dtype=np.int32) 
# marker[right_facets] = 1
# marker[left_facets] = 2
# marker[top_facets] = 3
# marker[bottom_facets] = 4
marker[front_facets] = 5
marker[back_facets] = 10
marker[air_facets] = 0

topology, cell_types, x = dolfinx.plot.vtk_mesh(domain, fdim, np.arange(np.max(num_cells_local)+1, dtype=np.int32))

p = pyvista.Plotter(window_size=[800, 800])
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Marker"] = marker
grid.set_active_scalars("Marker")
p.add_mesh(grid, show_edges=True)
if pyvista.OFF_SCREEN:
    figure = p.screenshot("subdomains_structured.png")
p.show()

num_cells_local = [    0     2     4 ... 48125 48126 48127]


Widget(value="<iframe src='http://localhost:34871/index.html?ui=P_0x7f33daf8d390_0&reconnect=auto' style='widt…

In [ ]:
air_facets.shape

(3712,)

In [ ]:
facet_tag.find(6)

array([44596, 44597, 44598, 44824, 44825, 45019, 45026, 45027, 45057,
       45058, 45360, 45361, 45374, 45375, 45570, 45571, 45584, 45585,
       45746, 45753, 45754, 45808, 45809, 45866, 45867, 46048, 46049,
       46076, 46077, 46114, 46115, 46229, 46230, 46257, 46258, 46295,
       46379, 46386, 46387, 46441, 46442, 46513, 46514, 46635, 46636,
       46663, 46664, 46719, 46787, 46788, 46815, 46816, 46912, 46919,
       46920, 46974, 46975, 47024, 47025, 47122, 47123, 47150, 47151,
       47194, 47246, 47247, 47274, 47275, 47347, 47354, 47355, 47404,
       47405, 47437, 47438, 47511, 47512, 47539, 47540, 47571, 47607,
       47608, 47635, 47636, 47684, 47691, 47692, 47724, 47725, 47752,
       47753, 47802, 47803, 47823, 47824, 47850, 47870, 47871, 47891,
       47892, 47923, 47930, 47931, 47941, 47942, 47962, 47963, 47995,
       47996, 48004, 48005, 48024, 48035, 48036, 48044, 48045, 48065,
       48066, 48074, 48075, 48095, 48096, 48103, 48110, 48111, 48119,
       48120, 48127]

In [ ]:
facet_tag.find(7)

array([   23,    27,    32, ..., 48091, 48092, 48104], dtype=int32)

In [ ]:
p_O2_in = 40                   # mmHg
front_dofs_O2 = fem.locate_dofs_topological(V.sub(0), model.facet_tag.dim, model.facet_tag.find(5)) # Gamma_in
bc_O2 = fem.dirichletbc(dolfinx.default_scalar_type(p_O2_in), front_dofs_O2, V.sub(0))

In [ ]:
front_dofs_O2

array([   0,    1,    2,   10,   12,   20,   22,   28,   32,   34,   42,
         46,   48,   62,   66,   68,   76,   82,   86,   88,   96,  104,
        108,  110,  128,  136,  140,  148,  154,  162,  172,  180,  188,
        210,  218,  224,  236,  242,  250,  264,  274,  282,  310,  320,
        326,  340,  348,  358,  376,  386,  394,  430,  440,  446,  464,
        474,  484,  504,  518,  524,  566,  578,  584,  606,  616,  628,
        652,  666,  672,  722,  734,  740,  776,  788,  828,  834,  898,
        904,  954, 1000, 1070], dtype=int32)

In [ ]:
front_dofs_CO2 = fem.locate_dofs_topological(V.sub(1), model.facet_tag.dim, model.facet_tag.find(5))

In [ ]:
front_dofs_CO2

array([   4,    5,    6,   11,   14,   21,   24,   29,   33,   36,   43,
         47,   50,   63,   67,   70,   77,   83,   87,   90,   97,  105,
        109,  112,  129,  137,  141,  149,  155,  163,  173,  181,  189,
        211,  219,  225,  237,  243,  251,  265,  275,  283,  311,  321,
        327,  341,  349,  359,  377,  387,  395,  431,  441,  447,  465,
        475,  485,  505,  519,  525,  567,  579,  585,  607,  617,  629,
        653,  667,  673,  723,  735,  741,  777,  789,  829,  835,  899,
        905,  955, 1001, 1071], dtype=int32)

In [ ]:
W = fem.FunctionSpace(domain, ("Lagrange", 1))

In [ ]:
# f1 = fem.Function(W)
# f2 = fem.Function(W)
# f1mf2 = fem.Expression(f1-f2, W.element.interpolation_points())

p_XY = fem.Function(V)

In [ ]:
alpha_CO2 = fem.Constant(domain, dolfinx.default_scalar_type(3.27E-5))             # M mmHg^-1
alpha_O2 = fem.Constant(domain, dolfinx.default_scalar_type(1.46E-6))              # M mmHg^-1

K2p = fem.Constant(domain, dolfinx.default_scalar_type(21.5))                      # M^-1
K2pp = fem.Constant(domain, dolfinx.default_scalar_type(1E-6))                     # M
K3p = fem.Constant(domain, dolfinx.default_scalar_type(11.3))                      # M^-1
K3pp = fem.Constant(domain, dolfinx.default_scalar_type(1E-6))                     # M
K5pp = fem.Constant(domain, dolfinx.default_scalar_type(2.63E-8))                  # M
K6pp = fem.Constant(domain, dolfinx.default_scalar_type(1.91E-8))                  # M

# Red blood cell pH (with pH_plasma = 7.4).
pH_rbc = 7.24
c_H = fem.Constant(domain, dolfinx.default_scalar_type(10**(-pH_rbc)))

# Variables defined in Dash & Bassingthwaighte (2016).

phi_1 = fem.Constant(domain, dolfinx.default_scalar_type(1 + (K2pp/c_H)))
phi_2 = fem.Constant(domain, dolfinx.default_scalar_type(1 + (K3pp/c_H)))
phi_3 = fem.Constant(domain, dolfinx.default_scalar_type(1 + (c_H/K5pp)))
phi_4 = fem.Constant(domain, dolfinx.default_scalar_type(1 + (c_H/K6pp)))

In [ ]:
P50_std = fem.Constant(domain, dolfinx.default_scalar_type(26.8))                  # mmHg
P_CO2_std = fem.Constant(domain, dolfinx.default_scalar_type(40))                  # mmHg
P50 = fem.Expression(P50_std + 1.273E-1*(p_CO2-P_CO2_std) + 1.083E-4*(p_CO2-P_CO2_std)**2,
                    V.sub(0).element.interpolation_points())

# Variable nH (Hill equation exponent), fit to data.
alpha = fem.Constant(domain, dolfinx.default_scalar_type(2.82))
beta = fem.Constant(domain, dolfinx.default_scalar_type(1.20))
gamma = fem.Constant(domain, dolfinx.default_scalar_type(29.25))

nH = fem.Expression(alpha - beta*10**(-p_O2/gamma),
                    V.sub(1).element.interpolation_points())
K4p = fem.Expression(((alpha_O2*p_O2)**(nH-1))*(K2p*alpha_CO2*p_CO2*phi_1 + phi_3)/(((alpha_O2*P50)**(nH))*(K3p*alpha_CO2*p_CO2*phi_2 + phi_4)),
                     V.sub())

# K_HbO2 = (K4p*(K3p*phi_2*alpha_CO2*P_CO2 + phi_4))
# K_HbO2 /= (K2p*phi_1*alpha_CO2*P_CO2 + phi_3)

# K_HbCO2 = (K2p*phi_1 + K3p*K4p*phi_2*alpha_O2*P_O2)
# K_HbCO2 /= (phi_3 + K4p*phi_4*alpha_O2*P_O2)

# S_HbO2 = (K_HbO2*alpha_O2*P_O2)/(1+(K_HbO2*alpha_O2*P_O2))
# S_HbCO2 = (K_HbCO2*alpha_CO2*P_CO2)/(1+(K_HbCO2*alpha_CO2*P_CO2))

TypeError: unsupported operand type(s) for -: 'Expression' and 'int'